In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
from pyspark.sql.functions import *
import pandas as pd
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/10/05 16:55:49 WARN Utils: Your hostname, SerennoMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.5.95 instead (on interface en0)
22/10/05 16:55:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/05 16:55:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [21]:
from keras.models import Sequential
from keras.layers import Dense
from numpy import array

In [3]:
tbl_transaction = spark.read.parquet("../data/curated/final_transaction.parquet")

In [58]:
tbl_transaction.dtypes

[('user_id', 'bigint'),
 ('merchant_abn', 'bigint'),
 ('dollar_value', 'double'),
 ('order_datetime', 'date'),
 ('whether_bigorder', 'int'),
 ('postcode', 'string'),
 ('average_prob_con', 'double'),
 ('whether_fraud', 'int')]

In [4]:
tbl_transaction.show(20)

tbl_transaction = tbl_transaction.filter("order_datetime >= '2021-02-28'")
tbl_transaction = tbl_transaction.filter("order_datetime <= '2022-08-28'")
tbl_transaction.agg(F.min("order_datetime"), F.max("order_datetime")).show()


+-------+------------+------------------+--------------+----------------+--------+-----------------+-------------+
|user_id|merchant_abn|      dollar_value|order_datetime|whether_bigorder|postcode| average_prob_con|whether_fraud|
+-------+------------+------------------+--------------+----------------+--------+-----------------+-------------+
|  11813| 11121775571|12.657354867135332|    2021-12-14|               0|    3959|9.132996419186014|            0|
|  11813| 49322182190|193.56810952479583|    2021-12-18|               0|    3959|9.132996419186014|            0|
|  11813| 11173270682| 165.3629765111036|    2021-10-06|               0|    3959|9.132996419186014|            0|
|  11813| 49322182190|   137.01318824084|    2022-07-25|               0|    3959|9.132996419186014|            0|
|  11813| 11173270682| 331.5640615872036|    2022-05-15|               0|    3959|9.132996419186014|            0|
|  11813| 56030772622| 209.8358230504878|    2021-08-28|               0|    395

+-------------------+-------------------+
|min(order_datetime)|max(order_datetime)|
+-------------------+-------------------+
|         2021-02-28|         2022-08-28|
+-------------------+-------------------+



In [5]:
#tbl_transaction.groupBy("merchant_abn").agg(F.count(F.col("order_datetime"))).show(20)
tbl_transaction.filter(F.col("merchant_abn")=="37935728745").orderBy("order_datetime").show()

+-------+------------+------------------+--------------+----------------+--------+------------------+-------------+
|user_id|merchant_abn|      dollar_value|order_datetime|whether_bigorder|postcode|  average_prob_con|whether_fraud|
+-------+------------+------------------+--------------+----------------+--------+------------------+-------------+
|   1125| 37935728745|1334.4788819571713|    2021-06-09|               0|    5073|14.921984314779792|            0|
|   4109| 37935728745|12036.874087756309|    2021-08-12|               1|    5381|24.360827467633634|            0|
|  19024| 37935728745|19184.728145204877|    2021-09-30|               1|    5351| 32.76628104964881|            0|
|  12890| 37935728745| 8471.766291167125|    2021-11-23|               0|    6017| 22.23238611786161|            0|
|   5577| 37935728745|1736.4273978943363|    2021-12-17|               0|    2425|12.216149932973964|            0|
|   2995| 37935728745|16087.205243232016|    2021-12-27|               1

In [8]:
#Xは18ヶ月分の月
#merchant_abnごとに
#月のaverageをとってデータにする:Y
#計算をする
from pyspark.sql.types import IntegerType
start_month = tbl_transaction.select(min("order_datetime").alias("min_date")).select(month("min_date").cast(IntegerType())).collect()[0][0]

from pyspark.sql.types import IntegerType
tbl_transaction2 = tbl_transaction\
    .withColumn('month',month(tbl_transaction.order_datetime).cast(IntegerType()))\
    .withColumn('year',year(tbl_transaction.order_datetime))
    
# set the start month to be 1, and calculate the number of months passed after that
transaction_month_num =  tbl_transaction2.withColumn('month_num',when(tbl_transaction2.year == "2022", F.col('month')-start_month+1+12).otherwise(F.col('month')-start_month+1))\
    .drop('month', 'year')


transaction_month_num.show()

+-------+------------+------------------+--------------+----------------+--------+-----------------+-------------+---------+
|user_id|merchant_abn|      dollar_value|order_datetime|whether_bigorder|postcode| average_prob_con|whether_fraud|month_num|
+-------+------------+------------------+--------------+----------------+--------+-----------------+-------------+---------+
|  11813| 11121775571|12.657354867135332|    2021-12-14|               0|    3959|9.132996419186014|            0|       11|
|  11813| 49322182190|193.56810952479583|    2021-12-18|               0|    3959|9.132996419186014|            0|       11|
|  11813| 11173270682| 165.3629765111036|    2021-10-06|               0|    3959|9.132996419186014|            0|        9|
|  11813| 49322182190|   137.01318824084|    2022-07-25|               0|    3959|9.132996419186014|            0|       18|
|  11813| 11173270682| 331.5640615872036|    2022-05-15|               0|    3959|9.132996419186014|            0|       16|


referenced cite: 
- https://bobrupakroy.medium.com/multi-layer-perception-time-series-8a1bddbb033a
- https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/
- https://codeit.us/blog/machine-learning-time-series-forecasting#:~:text=Time%20series%20forecasting%20in%20machine%20learning,-Before%20anything%20else&text=Time%20Series%20pertains%20to%20the,%22why%22%20behind%20its%20dataset.

In [20]:
#group each consecutive values of n months
def consec_data(data_list, n_months):
    X = []
    y = []
    for i in range(len(data_list)):
        end_month = i + n_months
        if end_month > len(data_list)-1:
            break
        get_x, get_y = data_list[i:end_month], data_list[end_month]
        X.append(get_x)
        y.append(get_y)
    return array(X), array(y)


Please install tensorflow

In [16]:

n_months = 3
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_months))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

2022-10-05 17:17:42.589315: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [44]:
data = transaction_month_num.select("merchant_abn", "dollar_value", "month_num").sort("merchant_abn", "month_num")
distinct_merchants = data.select("merchant_abn").distinct().rdd.flatMap(lambda x: x).collect()
MONTH_PERIOD = 18

predict_store = []
month = [m for m in range(1, MONTH_PERIOD+2)]
zeros = [0]*MONTH_PERIOD
month_list_initial = zip(month,zeros)
initial_df = spark.createDataFrame(data=month_list_initial, schema = ["month_num", "zeros"])
for merchant in distinct_merchants:
    one_merchant_data = data.filter(F.col("merchant_abn")==merchant)\
                            .groupBy(F.col("month_num"))\
                            .agg(F.avg("dollar_value").alias("avg_dollar_value"))
     
    one_merchant_and_missing_values = initial_df.join(one_merchant_data, initial_df.month_num == one_merchant_data.month_num, "left")\
                    .na.fill(value=0)\
                    .drop("zeros")   

    list_dollar = one_merchant_and_missing_values.select("avg_dollar_value").rdd.flatMap(lambda x: x).collect()
    X, y = consec_data(list_dollar, n_months)

    #fit
    model.fit(X, y, epochs=3000, verbose=0)
    #predict
    x_input = array(list_dollar[-n_months:])
    x_input = x_input.reshape((1, n_months))
    ypredict = model.predict(x_input, verbose=0)
    predict_store.append((merchant, ypredict[0][0]))
    

    

In [ ]:
save_df = spark.createDataFrame(data=predict_store, schema = ["merchant_abn","predict_dollar_value"])
save_df.to_parquet(f"../data/curated/predicted_dollar_value.parquet")

[(73256306726, 216.40903), (24406529929, 46.404495), (38700038932, 1133.9017)]
+------------+------------------+---------+
|merchant_abn|      dollar_value|month_num|
+------------+------------------+---------+
| 38700038932|1292.1571245407833|        1|
| 38700038932| 938.7308650806563|        1|
| 38700038932| 336.9310821210983|        1|
| 38700038932| 1700.092993601305|        1|
| 38700038932|1909.0415231377456|        1|
| 38700038932| 1001.077210662485|        1|
| 38700038932|  512.586476558845|        1|
| 38700038932| 644.5506173895864|        1|
| 38700038932|1353.9684306590711|        1|
| 38700038932|1008.0417171551393|        1|
| 38700038932|  363.962272793618|        1|
| 38700038932|1128.2709775543938|        2|
| 38700038932|1431.9753412409634|        2|
| 38700038932|1264.1416900941354|        2|
| 38700038932| 921.7906536652772|        2|
| 38700038932|1745.2925231901536|        2|
| 38700038932|1459.2458547449419|        2|
| 38700038932| 1186.008869610622|        